In [13]:
import os
import pandas as pd
import numpy as np

In [14]:
xlsx_paths = []
for path, subdirs, files in os.walk('..\data'):
    for name in files:
        if name.endswith('.xlsx'):
            xlsx_path = os.path.join(path, name)
            print(xlsx_path)
            xlsx_paths.append(xlsx_path)

..\data\Pojazdy wg wojewodztw w 2014 r._7282\Pojazdywgwojewodztw20141.xlsx
..\data\Pojazdy wg wojewodztw w 2015 r._2393\Pojazdywgwojewodztw20151.xlsx
..\data\Pojazdy wg wojewodztw w 2016 r._7427\Pojazdywgwojewodztw20161.xlsx
..\data\Pojazdy wg wojewodztw w 2017 r._7272\Pojazdywgwojewodztw20171.xlsx
..\data\Pojazdy wg wojewodztw w 2018 r._16845\Pojazdy_wg_wojew%C3%B3dztw_2018.xlsx
..\data\Pojazdy wg wojewodztw w 2019 r._29466\Pojazdy_wg_województw_2019.xlsx
..\data\Pojazdy wg wojewodztw w 2020 r._29470\Pojazdy_wg_województw_2020.xlsx
..\data\Pojazdy wg wojewodztw w 2021 r._36007\Pojazdy_wg_województw_2021.xlsx
..\data\Pojazdy zarejestrowane w roku 2022 wg wojewodztw_43031\Pojazdy_wg_województw_2022.xlsx


In [15]:
import re
def get_year_from_path(file_path: str):
    result = re.search(r" (\d{4}) ", file_path)
    return result.group(1)

In [16]:
for path in xlsx_paths:
    print(get_year_from_path(path))

2014
2015
2016
2017
2018
2019
2020
2021
2022


In [17]:
def parse_xlsx(xlsx_path: str):
    df = pd.ExcelFile(xlsx_path).parse()
    df.columns= df.columns.str.lower()
    df['województwo'] = df['województwo'].str.lower()
    df = df.drop(columns=['teryt'], errors='ignore')
    df['year'] = get_year_from_path(xlsx_path)
    df = df.replace(np.nan,None,regex = True)
    new_column_names = ['voivodeship'] + list(range(1,13)) + ['year']
    df.columns = new_column_names
    df = df.melt(id_vars =['voivodeship', 'year'], value_vars =list(range(1,13)),
            var_name ='month', value_name ='vehicles')

    return df

In [18]:
df_list = []
for xlsx_path in xlsx_paths:
    df_list.append(parse_xlsx(xlsx_path))

In [19]:
full_df = pd.concat(df_list)

In [20]:
full_df['date'] = full_df['month'].astype(str).str.zfill(2) + "-" + full_df['year'].astype(str)

In [21]:
full_df.voivodeship = full_df.voivodeship.apply(lambda x: 'warmińsko-mazurskie' if 'warmińskie-mazurskie' in x else x)

In [22]:
full_df

,voivodeship,year,month,vehicles,date
0,dolnośląskie,2014,1,7921,01-2014
1,kujawsko-pomorskie,2014,1,5153,01-2014
2,lubelskie,2014,1,4864,01-2014
3,lubuskie,2014,1,3217,01-2014
4,łódzkie,2014,1,5798,01-2014
...,...,...,...,...,...
187,śląskie,2022,12,None,12-2022
188,świętokrzyskie,2022,12,None,12-2022
189,warmińsko-mazurskie,2022,12,None,12-2022
190,wielkopolskie,2022,12,None,12-2022


In [23]:
full_df.to_csv('../output/out.csv')